In [1]:
import pandas as pd
from glob import glob
from pypcleaner import convert_excel_time

In [2]:
df = pd.read_hdf('big_multi_index.h5')
scans = pd.read_pickle('scans.p')
imaging_id_codes = pd.read_pickle('imaging_id_codes.p')
id_codes = imaging_id_codes.loc[scans.index,'prefix']
scan_id_df = df.loc[scans.index]
scan_id_df.drop('day_0', axis='columns', level=0, inplace = True)

drop_these = ['bp_three_hr','bp_three_hr_systolic','bp_three_hr_diastolic','bp_three_hr_datetime',
              'bp_seven_hr','bp_seven_hr_systolic','bp_seven_hr_diastolic','bp_seven_hr_datetime',
              'bp_other_hr','bp_other_hr_systolic','bp_other_hr_diastolic','bp_other_hr_datetime',
              'hrt_seven_hr','hrt_seven_hr_rate','hrt_seven_hr_datetime','hrt_other_hr',
              'hrt_other_hr_rate','hrt_other_hr_datetime','surgical_intervention',
              'surgical_intervention_detail','invasive_intervention','invasive_intervention_detail',
              'imaging_intervention','imaging_intervention_detail',
              'other_intervention','other_intervention_detail','medication_given','medication_comment']

scan_id_df.drop([('day_baseline','recent_medical_history',c) for c in drop_these], axis = 'columns',inplace = True)

scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')] = \
scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')].apply(convert_excel_time).dt.round('5min')

In [3]:
remove_tests = \
[('day_baseline','physical_examination'), 
 ('day_baseline', 'past_medical_history'), 
 ('day_baseline', 'CCMI'), 
 ('day_baseline', 'further_stroke_event_part'), 
 ('day_baseline', 'bloods'), 
 ('day_baseline', 'laboratory_tests'), 
 ('day_baseline', 'radiological_scans'), 
 ('day_baseline', 'concomitant_medications'), 
 ('day_baseline', 'pre-stroke_mRS'),
 ('day_7', 'bloods') , 
 ('day_7', 'history_of_depression'), 
 ('day_7', 'physical_risk_factors'), 
 ('day_7', 'diet_q'),
 ('day_7', 'laboratory_tests'), 
 ('day_7', 'concomitant_medications'), 
 ('day_7', 'concomitant_medications_plus'), 
 ('day_90', 'bloods'), 
 ('day_90', 'physical_examination'), 
 ('day_90', 'physical_risk_factors'), 
 ('day_90', 'diet_q'), 
 ('day_90', 'laboratory_tests'), 
 ('day_90', 'further_stroke_event_part'), 
 ('day_90', 'history_of_depression'), 
 ('day_90', 'concomitant_medications'), 
 ('day_365', 'bloods'), 
 ('day_365', 'physical_examination'), 
 ('day_365', 'physical_risk_factors'), 
 ('day_365', 'diet_q'), 
 ('day_365', 'laboratory_tests'), 
 ('day_365', 'further_stroke_event_part'), 
 ('day_365', 'history_of_depression'), 
 ('day_365', 'concomitant_medications'), 
 ('day_1', 'vital_signs'), 
 ('day_1', 'bloods'), 
 ('day_1', 'laboratory_tests'), 
 ('day_1', 'serious_adverse_event')]

scan_id_df.drop(remove_tests, axis = 'columns',inplace = True)

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [4]:
demographics = pd.read_pickle('base_df_data.p')


stroke_onset = demographics.loc[scans.index,['identity_stroke_onset']]
multi_NIHSS = scan_id_df[('day_baseline','NIHSS_multiple')]
multi_with_stroke_onset = stroke_onset.join(multi_NIHSS)

for c in ['nihss_three_hr_datetime', 'nihss_seven_hr_datetime', 
          'nihss_eighteen_hr_datetime', 'nihss_other_hr_datetime']:
    hrs = multi_with_stroke_onset[c] - multi_with_stroke_onset['identity_stroke_onset']
    multi_with_stroke_onset[c] = hrs.dt.round('5min')
    
scan_id_df[('day_baseline','NIHSS_multiple')] = multi_with_stroke_onset

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


In [5]:
bit_1_cols = demographics.loc[:,'identity_gender':'identity_age_at_stroke_in_years'].columns
bit_2_cols = demographics.loc[:,'demographics_ancestry_1':].columns
useful_columns = bit_1_cols.append(bit_2_cols)
demographics = demographics.loc[scans.index,useful_columns]

In [6]:
demographics['identity_gender'] = demographics['identity_gender'].map({1:'female',0:'male'})

In [11]:
id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,'prefix']
infarct_type = scan_id_df[('day_1','infarct_type')]
scan_id_df[('day_90','lesion_location_confirmed')] = \
scan_id_df[('day_90','lesion_location_confirmed')].join([id_codes,infarct_type])

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


This doesn't work! Sort out. Probably need to create multi index for ones to join

In [12]:
scan_id_df[('day_90','lesion_location_confirmed')]

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,lesion location confirmed by imaging at 3 months (BC report)
119,right
123,right
124,left
126,right
127,right
129,right
132,right
133,left pons
136,right
137,right


### Do it!

delete the individual columns and keep the calculated average

In [ ]:
scan_id_df[('day_90','TUGT_average','correct_average')] = \
scan_id_df['day_90']['TUGT_average'][['time1', 'time2', 'time3']].mean(axis = 'columns').round(1)
scan_id_df[('day_365','TUGT','correct_average')] = \
scan_id_df['day_365']['TUGT'][['time1', 'time2', 'time3']].mean(axis='columns').round(1)

bad = \
[('day_90', 'TUGT', 'time_taken'),
 ('day_90', 'TUGT_average', 'time1'),
 ('day_90', 'TUGT_average', 'time2'),
 ('day_90', 'TUGT_average', 'time3'),
 ('day_90', 'TUGT_average', 'TUGT_average'),
 ('day_365', 'TUGT', 'time1'),
 ('day_365', 'TUGT', 'time2'),
 ('day_365', 'TUGT', 'time3'),
 ('day_365', 'TUGT', 'average')]
scan_id_df.drop(bad,axis='columns',inplace=True)

In [ ]:
# drop mri
mri_drop = scan_id_df.filter(regex='MRI').columns.tolist()
scan_id_df.drop(mri_drop,axis='columns',inplace=True)

In [ ]:
response_drop = \
[('day_7', 'RAPA', 'resp'),
 ('day_7', 'RAPA', 'resp_other'),
 ('day_7', 'RAPA', 'resp_date'),
 ('day_90', 'mRS', 'resp'),
 ('day_90', 'mRS', 'resp_other'),
 ('day_90', 'BI', 'resp.1'),
 ('day_90', 'BI', 'resp_other.1'),
 ('day_90', 'SIS', 'resp.2'),
 ('day_90', 'SIS', 'resp_other.2'),
 ('day_90', 'RAPA', 'resp'),
 ('day_90', 'RAPA', 'resp_other'),
 ('day_90', 'RAPA', 'resp_date'),
 ('day_90', 'WSAS', 'resp.5'),
 ('day_90', 'WSAS', 'resp_other.5'),
 ('day_90', 'WSAS', 'resp_date.2'),
 ('day_365', 'mRS', 'resp'),
 ('day_365', 'mRS', 'resp_other'),
 ('day_365', 'BI', 'resp.1'),
 ('day_365', 'BI', 'resp_other.1'),
 ('day_365', 'SIS', 'resp.2'),
 ('day_365', 'SIS', 'resp_other.2'),
 ('day_365', 'RAPA', 'resp'),
 ('day_365', 'RAPA', 'resp_other'),
 ('day_365', 'RAPA', 'resp_date'),
 ('day_365', 'WSAS', 'resp.5'),
 ('day_365', 'WSAS', 'resp_other.5'),
 ('day_365', 'WSAS', 'resp_date.2')]

scan_id_df.drop(response_drop,axis='columns',inplace=True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [ ]:
drop_nihss_multiple = [('day_baseline','NIHSS_multiple',c) for c in ['nihss_three_hr',
                                                                     'nihss_seven_hr',
                                                                     'nihss_eighteen_hr',
                                                                     'nihss_other_hr']]

scan_id_df.drop(drop_nihss_multiple,axis = 'columns',inplace= True)

In [ ]:
# RAVENs had no data
orig_df = pd.read_excel('data/extract/START 12 months (20161121).xlsx')
ravens = orig_df.iloc[scans.index,689:762]
ravens.rename(columns = {'RAVENS_total_score_12mth':'score'}, inplace = True)
scan_id_df[('day_365','RAVENs')] = ravens

In [ ]:
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [13]:
scan_id_df

day_baseline                                               \
    NIHSS_multiple                                                
    nihss_three_hr nihss_three_hr_score nihss_three_hr_datetime   
119            NaN                  NaN                     NaT   
123            NaN                  NaN                     NaT   
124            NaN                  NaN                     NaT   
126            NaN                  NaN                     NaT   
127            NaN                  NaN                     NaT   
129              1                    5         0 days 01:30:00   
132              1                    2         0 days 02:30:00   
133              0                  NaN                     NaT   
136              0                  NaN                     NaT   
137              0                  NaN                     NaT   
138              0                  NaN                     NaT   
139              0                  NaN                     NaT   
140              0                  NaN                     NaT   
141              0                  NaN                     NaT   
142              0                  NaN                     NaT   
143              0                  NaN                     NaT   
144              0                  NaN                     NaT   
146              0                  NaN                     NaT   
147              0                  NaN                     NaT   
148              0                  NaN                     NaT   
151              0                  NaN                     NaT   
152              0                  NaN                     NaT   
154              0                  NaN                     NaT   
155              0                  NaN                     NaT   
156              0                  NaN                     NaT   
157              0                  NaN                     NaT   
159              0                  NaN                     NaT   
161              0                  NaN                     NaT   
162              0                  NaN                     NaT   
163              0                  NaN                     NaT   
..             ...                  ...                     ...   
178            NaN                  NaN                     NaT   
180              0                  NaN                     NaT   
182              1                    0         0 days 04:55:00   
183              0                  NaN                     NaT   
186              0                  NaN                     NaT   
188              1                   12         0 days 00:55:00   
189              1                    4         0 days 01:15:00   
190              0                  NaN                     NaT   
191              1                    1         0 days 01:30:00   
192              0                  NaN                     NaT   
194              0                  NaN                     NaT   
196              1                   17         0 days 02:30:00   
197              0                  NaN                     NaT   
198              1                    2         0 days 01:35:00   
199              0                  NaN                     NaT   
200              0                  NaN                     NaT   
201              0                  NaN                     NaT   
202              0                  NaN                     NaT   
204              1                    2         0 days 01:05:00   
205              1                    1         0 days 02:30:00   
206              1                    4         0 days 00:45:00   
207              1                    2         0 days 10:30:00   
208              0                  NaN                     NaT   
209              1                    8         0 days 03:00:00   
210              0                  NaN                     NaT   
211              0                  NaN              